# CONSTRUCTION OF FX VOL SURFACE - PART I

In this notebook I show how you can calibrate to the FX Vol Surface to ATM, 25D MS and 25D RR at one expiry date and analyse different volatility interpolation methods.

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from financepy.utils import *

In [6]:
from financepy.models.black_scholes import *
from financepy.products.fx import *
from financepy.market.curves import DiscountCurveFlat
from financepy.market.volatility import * 

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Internal error at <numba.core.typeinfer.CallConstraint object at 0x00000217A4282A90>.

Enable logging at debug level for details.
[1m
File "..\..\..\..\..\..\..\..\..\AppData\Roaming\Python\Python39\site-packages\financepy\market\volatility\fx_vol_surface_plus.py", line 758:[0m
[1mdef _solver_for_smile_strike(s, t, rd, rf,
    <source elided>

[1m    K = newton_secant(_delta_fit, x0=initialGuess, args=argtuple,
[0m    [1m^[0m[0m


## Load the Volatility Market Quotes

In [ ]:
valuation_date = Date(10, 4, 2020)

In [ ]:
forName = "EUR"
domName = "USD"
forCCRate = 0.03460  # EUR
domCCRate = 0.02940  # USD

We need to set up the option details - what you would find in the Term Sheet.

In [ ]:
dom_discount_curve = DiscountCurveFlat(valuation_date, domCCRate)
for_discount_curve = DiscountCurveFlat(valuation_date, forCCRate)

In [ ]:
currency_pair = forName + domName
spot_fx_rate = 1.3465        

I now load the option tenor, the ATM vol and the market strangle and risk-reversal quotes.

In [ ]:
tenors = ['1Y']
atm_vols = [18.250]
ms25DeltaVols = [0.95]
rr25DeltaVols = [-0.60]

We need to set some parameters for the vol surface.

In [ ]:
notional_currency = forName
atmMethod = FinFXATMMethod.FWD_DELTA_NEUTRAL
deltaMethod = FinFXDeltaMethod.SPOT_DELTA

In [ ]:
fxVolSurfaceClark = FXVolSurface(valuation_date, spot_fx_rate, currency_pair, notional_currency,
                                   dom_discount_curve, for_discount_curve,
                                   tenors, atm_vols, ms25DeltaVols, rr25DeltaVols,
                                   atmMethod, deltaMethod, VolFunctionTypes.CLARK5)

In [ ]:
fxVolSurfaceSABR = FXVolSurface(valuation_date, spot_fx_rate, currency_pair, notional_currency,
                                   dom_discount_curve, for_discount_curve,
                                   tenors, atm_vols, ms25DeltaVols, rr25DeltaVols,
                                   atmMethod, deltaMethod, VolFunctionTypes.SABR)

In [ ]:
fxVolSurfaceBBG = FXVolSurface(valuation_date, spot_fx_rate, currency_pair, notional_currency,
                                  dom_discount_curve, for_discount_curve,
                                  tenors, atm_vols, ms25DeltaVols, rr25DeltaVols,
                                  atmMethod, deltaMethod, VolFunctionTypes.BBG)

We can examine the calibration

In [ ]:
fxVolSurfaceClark.check_calibration(True)

In [ ]:
fxVolSurfaceSABR.check_calibration(True)

In [ ]:
fxVolSurfaceBBG.check_calibration(True)

## Volatility Smile Analysis - Different Volatility Function Types

In [ ]:
strikes = np.linspace(0.5, 2.5, 1000)

In [ ]:
expiry_date = valuation_date.add_tenor("1Y")

In [ ]:
volsClark = []
volsSABR = []
volsBBG = []

for k in strikes:
    volClark = fxVolSurfaceClark.volatility(k, expiry_date)
    volSABR = fxVolSurfaceSABR.volatility(k, expiry_date)
    volBBG = fxVolSurfaceBBG.volatility(k, expiry_date)
    volsClark.append(volClark*100.0)    
    volsSABR.append(volSABR*100.0)    
    volsBBG.append(volBBG*100.0)    

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(strikes, volsClark, label="Clark")
plt.plot(strikes, volsSABR, label="SABR")
plt.plot(strikes, volsBBG, label="BBG")
plt.xlabel("Strike")
plt.ylabel("Black Scholes Volatility (%)")
plt.title("Comparison of Volatility Smiles")
plt.legend();

## Implied FX Rate Probability Density Functions

In [ ]:
lower = 0.50
upper = 2.25
dbnClark = fxVolSurfaceClark.implied_dbns(lower, upper, 1000)
dbnSABR = fxVolSurfaceSABR.implied_dbns(lower, upper, 1000)
dbnBBG = fxVolSurfaceBBG.implied_dbns(lower, upper, 1000)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(dbnClark[0]._x, dbnClark[0]._densitydx, label="Clark")
plt.plot(dbnSABR[0]._x, dbnSABR[0]._densitydx, label="SABR")
plt.plot(dbnBBG[0]._x, dbnBBG[0]._densitydx, label="BBG")
plt.title("Implied Probability Density Function")
plt.legend();

## Expiry Date Interpolation

We only have one expiry date. The volatility is therefore assumed to be flat at the level of this expiry date.

In [ ]:
k = 1.30

In [ ]:
years = np.linspace(0.0, 2.0, 100)

In [ ]:
expiry_dates = valuation_date.add_years(years)

In [ ]:
volsClark = []
volsSABR = []
volsBBG = []

for expiry_date in expiry_dates:
    
    volClark = fxVolSurfaceClark.volatility(k, expiry_date)
    volSABR = fxVolSurfaceSABR.volatility(k, expiry_date)
    volBBG = fxVolSurfaceBBG.volatility(k, expiry_date)

    volsClark.append(volClark*100.0)    
    volsSABR.append(volSABR*100.0)    
    volsBBG.append(volBBG*100.0)    

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(years, volsClark, label="Clark")
plt.plot(years, volsSABR, label="SABR")
plt.plot(years, volsBBG, label="BBG")
plt.xlabel("Years")
plt.ylabel("Black Scholes Volatility (%)")
plt.title("Comparison of Volatility Time Interpolation")
plt.legend();

Copyright (c) 2020, Dominic O'Kane 